In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import liana as li
import anndata
import scanpy as sc
import os 
import scipy

In [ ]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

# !pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
# !pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
# !pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

# Helper function for visualization.
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from utils import *

1.13.1+cu117
1.13.1+cu117


In [ ]:
import argparse
import os.path as osp

import torch
import torch.nn.functional as F

from torch_geometric.datasets import Entities
from torch_geometric.nn import FastRGCNConv, RGCNConv, GCNConv, InnerProductDecoder, GAE, VGAE
from torch_geometric.utils import k_hop_subgraph

from torch_geometric.datasets import Planetoid
from torch_geometric.data.data import Data
from torch_geometric.transforms import NormalizeFeatures


from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import SpectralClustering
import random
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

from sklearn.metrics.pairwise import cosine_similarity

from scipy.io import mmread

from scipy.stats import wilcoxon
import anndata
import scanpy as sc

## Processing with csv file

In [ ]:
matrix = pd.read_csv("../../data/GraphComm_Input/Pre_Post/GSE138267/GSM4104147_SC39.LB17019/matrix.csv",index_col=0)

In [ ]:
adata = anndata.AnnData(matrix.transpose())

In [ ]:
adata

In [ ]:
sc.pp.filter_genes(adata, min_cells=5)
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
sc.pp.neighbors(adata)
sc.tl.leiden(adata)

In [ ]:
meta = pd.DataFrame({"cell":adata.obs["leiden"].index.tolist(),"labels":adata.obs["leiden"].tolist()})

In [ ]:
meta.index = meta["cell"].tolist()

In [ ]:
adata.obs = meta

In [ ]:
matrix = pd.DataFrame(adata.X.transpose(),columns=adata.obs.index.tolist(),index=adata.var.index.tolist())

# Processing with h5ad

In [ ]:

adata = sc.read_h5ad("/h/soemily/new_GAT/data/raw_data/Cardiac_cells/Visium-RZ_FZ_P5.h5ad")
matrix = pd.DataFrame.sparse.from_spmatrix(adata.X,index=adata.obs.index.tolist(),columns=adata.var["feature_name"].tolist())
sc.pp.filter_genes(adata, min_cells=10)
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
sc.pp.neighbors(adata)
sc.tl.leiden(adata)
matrix = pd.DataFrame.sparse.from_spmatrix(adata.X,index=adata.obs.index.tolist(),columns=adata.var["feature_name"].tolist())
meta = pd.DataFrame({"cell":adata.obs.index.tolist(),"labels":adata.obs["cell_type_original"].tolist()})
meta.index = meta["cell"].tolist()
#removing lymphoid as it is only one cell, cannot be used in benchmarking
meta = meta[meta['labels'] != "Lymphoid"]
adata = adata[meta["cell"].tolist()]
matrix = matrix.transpose()

matrix = matrix[meta.index.tolist()]

index = matrix.index.tolist()
adata.obs = meta


# rest of preprocessing

In [ ]:
#sc.tl.rank_genes_groups(adata, 'labels')
cell_groups = meta['labels'].unique().tolist()
matrix_list = {}
for i in cell_groups:
    cells = meta[meta["labels"]==i].index.tolist()
    temp_matrix = matrix[cells]
    matrix_list[i] = (temp_matrix.mean(axis=1)[temp_matrix.mean(axis=1) >= 0].index.tolist())

In [ ]:
adata = adata.transpose()

In [ ]:
cell_type_df = matrix_list

In [ ]:
#mean_expression = np.mean(matrix.values[matrix.values != 0])
nodes = pd.DataFrame({"category":[],"identifier":[]})

In [ ]:
LR_nodes = pd.read_csv("/data/LR_database/new_OmniPath_nodes.csv",index_col=0)
Omnipath_network = pd.read_csv("/data/LR_database/new_OmniPath_interactions.csv",index_col=0)

In [145]:
ligands = LR_nodes[LR_nodes["category"]=="Ligand"]["identifier"].tolist()
receptors = LR_nodes[LR_nodes["category"]=="Receptor"]["identifier"].tolist()

In [146]:
ligand_list = []
receptor_list = []
new_cell_df = {}
for i in cell_type_df.keys():
    ligand_list.extend(list(set(ligands) & set(cell_type_df[i])))
    receptor_list.extend(list(set(receptors) & set(cell_type_df[i])))
    new_cell_df[i] = [list(set(ligands) & set(cell_type_df[i])),list(set(receptors) & set(cell_type_df[i]))]

In [147]:
for i in new_cell_df.keys():
    new_cell_df[i][0] = [j+"_Ligand" for j in new_cell_df[i][0]]
    new_cell_df[i][1] = [j+"_Receptor" for j in new_cell_df[i][1]]

In [148]:
ligand_list = list(set(ligand_list))
receptor_list = list(set(receptor_list))

In [149]:
ligand_list = [i+"_Ligand" for i in ligand_list]
receptor_list = [i+"_Receptor" for i in receptor_list]

In [150]:
nodes = pd.concat([nodes,pd.DataFrame({"category":["Cell Group"] * len(list(cell_type_df.keys())), "identifier":list(cell_type_df.keys())})])
nodes = pd.concat([nodes,pd.DataFrame({"category":["Ligand"] * len(ligand_list), "identifier":ligand_list})])
nodes = pd.concat([nodes,pd.DataFrame({"category":["Receptor"] * len(receptor_list), "identifier":receptor_list})])

In [151]:
new_identifier = [row["identifier"] + "_" + row["category"] for index,row in LR_nodes.iterrows()]

In [152]:
LR_nodes["identifier"] = new_identifier

In [153]:
nodes["Id"] = range(0,nodes.shape[0])
nodes = nodes[["Id","category","identifier"]]

In [154]:
nodes.index = nodes.index.astype('int')
nodes["Id"] = nodes["Id"].astype('int')

In [155]:
meta.index = meta["cell"].tolist()

In [156]:
interactions = pd.DataFrame({"Src":[],"Dst":[],"Weight":[],"edge_type":[]})

In [157]:
LR_nodes.index = LR_nodes["Id"].tolist()

In [158]:
Omnipath_network["Src"] = LR_nodes.loc[Omnipath_network["Src"].tolist()]["identifier"].tolist()
Omnipath_network["Dst"] = LR_nodes.loc[Omnipath_network["Dst"].tolist()]["identifier"].tolist()


In [159]:
source_list = []
dest_list = []
weight_list = []
edge_type_list = []
for i in new_cell_df.keys():
    source_list.extend([i] * (len(new_cell_df[i][0]) + len(new_cell_df[i][1])))
    dest_list.extend(new_cell_df[i][0])
    dest_list.extend(new_cell_df[i][1])
    weight_list.extend([1] * (len(new_cell_df[i][0]) + len(new_cell_df[i][1])))
    edge_type_list.extend([1] * (len(new_cell_df[i][0]) + len(new_cell_df[i][1])))

In [160]:
interactions["Src"] = source_list
interactions["Dst"] = dest_list
interactions["Weight"] = weight_list
interactions["edge_type"] = edge_type_list

In [161]:
nodes.index = nodes["identifier"].tolist()

In [162]:
nodes = nodes.drop_duplicates("identifier")
nodes["Id"] = range(0,nodes.shape[0])

In [163]:
interactions["Src"] = nodes.loc[interactions["Src"].tolist()]["Id"].tolist()
interactions["Dst"] = nodes.loc[interactions["Dst"].tolist()]["Id"].tolist()

In [164]:
first_data,first_nodes,first_interactions = make_dataset(nodes,interactions,first=False,pathway_encode=False)

# Edit the ground truth to only contain information from this dataset

In [165]:
LR_nodes.index = LR_nodes["identifier"].tolist()

In [166]:
#LR_nodes["identifier"] = [row["identifier"] + "_" + row["category"] for index,row in LR_nodes.iterrows()]
LR_nodes = LR_nodes[(LR_nodes["identifier"].isin(ligand_list)) | (LR_nodes["identifier"].isin(receptor_list))]
Omnipath_network = Omnipath_network[(Omnipath_network["Src"].isin(LR_nodes["identifier"].tolist())) & (Omnipath_network["Dst"].isin(LR_nodes["identifier"].tolist()))]

In [167]:
ligand_list = Omnipath_network["Src"].tolist()
receptor_list = Omnipath_network["Dst"].tolist()
LR_nodes = LR_nodes[(LR_nodes["identifier"].isin(ligand_list)) | (LR_nodes["identifier"].isin(receptor_list))]
Omnipath_network = Omnipath_network[(Omnipath_network["Src"].isin(LR_nodes["identifier"].tolist())) & (Omnipath_network["Dst"].isin(LR_nodes["identifier"].tolist()))]

In [168]:
ligand_list = Omnipath_network["Src"].tolist()
receptor_list = Omnipath_network["Dst"].tolist()

In [169]:
LR_nodes["Id"] = range(0,LR_nodes.shape[0])
LR_nodes.index = LR_nodes["identifier"].tolist()
Omnipath_network["Src"] = LR_nodes.loc[Omnipath_network["Src"].tolist()]["Id"].tolist()
Omnipath_network["Dst"] = LR_nodes.loc[Omnipath_network["Dst"].tolist()]["Id"].tolist()

In [170]:
interactions["edge_type"]=1

In [ ]:
pd.read_csv("../..")

In [77]:
os.system("mkdir -p /h/soemily/new_GAT/data/GraphComm_Input/Drosophila")

0

In [119]:
nodes

,Id,category,identifier
0,0,Cell Group,0
13,1,Cell Group,13
15,2,Cell Group,15
6,3,Cell Group,6
7,4,Cell Group,7
...,...,...,...
RIOK2_Receptor,433,Receptor,RIOK2_Receptor
SLU7_Receptor,434,Receptor,SLU7_Receptor
FEN1_Receptor,435,Receptor,FEN1_Receptor
RIF1_Receptor,436,Receptor,RIF1_Receptor


In [44]:
nodes.to_csv("/h/soemily/new_GAT/data/GraphComm_Input/Drosophila/nodes.csv")
interactions.to_csv("/h/soemily/new_GAT/data/GraphComm_Input/Drosophila/interactions.csv")
meta.to_csv("/h/soemily/new_GAT/data/GraphComm_Input/Drosophila/meta.csv")
#matrix.to_csv("/h/soemily/new_GAT/data/GraphComm_Input/Drosophila/matrix.csv")
LR_nodes.to_csv("/h/soemily/new_GAT/data/GraphComm_Input/Drosophila/LR_nodes.csv")
Omnipath_network.to_csv("/h/soemily/new_GAT/data/GraphComm_Input/Drosophila/Omnipath_network.csv")

In [45]:
matrix

,TCCCTAAAGTAN,TTTAAGCTCTTN,AGAGAGAATACA,GCCCGTGGAGCA,ACTAGACCAAGT,CACTTAATCTAC,CGCCCTTACTAC,CCTTTTATGGAC,GGAATAGACGGT,AGCTATGACATT,...,TCTGTTTCTGGT,TATAGGCCCTGA,GACGGATAACCG,AACGTTCCTCTC,AACATTCGAAAA,ACATTTGAACGT,TTAGTCACATAT,CCAACAGTACTN,AATTATCGCCTA,GCGACACTCTAC
NAN,0.027214,0.061960,0.000000,0.000000,0.024669,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CG1463,0.243722,0.140006,0.256528,0.178183,0.163851,0.362570,0.245377,0.149403,0.107093,0.138805,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CG15706,0.000000,0.115672,0.000000,0.052641,0.047901,0.062303,0.130457,0.034702,0.074277,0.061500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HIS3.3B,1.079375,1.069833,1.046530,1.066682,1.057468,1.047348,1.120534,1.042432,1.104816,1.003525,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IX,0.160185,0.089723,0.131374,0.052641,0.163851,0.090161,0.319742,0.096247,0.038722,0.161443,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CG43156,0.052724,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RIC,0.052724,0.162892,0.048766,0.052641,0.110294,0.062303,0.000000,0.066664,0.074277,0.089019,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SA,0.052724,0.089723,0.025113,0.052641,0.069837,0.032347,0.083140,0.123744,0.107093,0.089019,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MRPL10,0.076710,0.089723,0.092247,0.195587,0.110294,0.062303,0.172715,0.000000,0.192317,0.061500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
